In [2]:
import pandas as pd
import numpy as np
def read_data(filename):
    df = pd.read_csv(filename, sep='\t')
    return (df)


In [16]:
import helper
import submission as submission
input_data = read_data('./tests/07_test.txt')
output = submission.buc_rec_optimized(input_data)
output


<class 'pandas.core.frame.DataFrame'>


In [14]:
output_data = submission.buc_rec_optimized(input_data).astype(str).reset_index(drop=True).astype(str)
output_data

,A,B,C,M
0,1,1,1,10
1,1,1,2,20
2,1,1,ALL,30
3,1,2,1,30
4,1,2,ALL,30
5,1,3,1,40
6,1,3,ALL,40
7,1,ALL,1,80
8,1,ALL,2,20
9,1,ALL,ALL,100


In [155]:
input_data = pd.DataFrame({'A': [1],
                           'B': [2],
                           'C': [3],
                           'D': [4],
                           'M': [100]})
input_data


,A,B,C,D,M
0,1,2,3,4,100


In [156]:
import helper
import submission as submission
output = submission.buc_rec_optimized(input_data)
output

,A,B,C,D,M
0,1,2,3,4,100
1,1,2,3,ALL,100
2,1,2,ALL,4,100
3,1,2,ALL,ALL,100
4,1,ALL,3,4,100
5,1,ALL,3,ALL,100
6,1,ALL,ALL,4,100
7,1,ALL,ALL,ALL,100
8,ALL,2,3,4,100
9,ALL,2,3,ALL,100


In [80]:
def buc_rec(input):
    # Note that input is a DataFrame
    dims = input.shape[1]
    
    if dims == 1:
        # only the measure dim
        input_sum = sum( project_data(input, 0) )
        return input_sum
    else:
        # the general case
        dim0_vals = set(project_data(input, 0).values)
        dict1 = {}
        for dim0_v in dim0_vals:
            sub_data = slice_data_dim0(input, dim0_v)
            num = buc_rec(sub_data)
            dict1[dim0_v] = num
        ## for R_{ALL}
        sub_data = remove_first_dim(input)
        num = buc_rec(sub_data)
        dict1['ALL'] = num
    
    
    
    return dict1

In [81]:
input = buc_rec(info)
input

{1: {2: {100: 200, 'ALL': 200}, 'ALL': {100: 200, 'ALL': 200}},
 2: {1: {20: 400, 'ALL': 400}, 'ALL': {20: 400, 'ALL': 400}},
 'ALL': {1: {20: 400, 'ALL': 400},
  2: {100: 200, 'ALL': 200},
  'ALL': {100: 200, 20: 400, 'ALL': 600}}}

In [140]:
def print_dict(input, list1):
    output = []
    if isinstance(input, dict):
        for key in input:
            temp = list1
            temp = temp + [key]
            output = output + print_dict(input[key], temp)
    else:
        temp = list1 + [input]
        output = [temp]
    
    return output
        

In [143]:
list1 = []
tt = print_dict(input, list1)
tt

[[1, 2, 100, 200],
 [1, 2, 'ALL', 200],
 [1, 'ALL', 100, 200],
 [1, 'ALL', 'ALL', 200],
 [2, 1, 20, 400],
 [2, 1, 'ALL', 400],
 [2, 'ALL', 20, 400],
 [2, 'ALL', 'ALL', 400],
 ['ALL', 1, 20, 400],
 ['ALL', 1, 'ALL', 400],
 ['ALL', 2, 100, 200],
 ['ALL', 2, 'ALL', 200],
 ['ALL', 'ALL', 100, 200],
 ['ALL', 'ALL', 20, 400],
 ['ALL', 'ALL', 'ALL', 600]]

In [148]:
output = pd.DataFrame(tt, columns = info.columns)
output

,A,B,C,M
0,1,2,100,200
1,1,2,ALL,200
2,1,ALL,100,200
3,1,ALL,ALL,200
4,2,1,20,400
5,2,1,ALL,400
6,2,ALL,20,400
7,2,ALL,ALL,400
8,ALL,1,20,400
9,ALL,1,ALL,400


In [147]:
print(info.columns)

Index(['A', 'B', 'C', 'M'], dtype='object')


In [1]:
def buc_rec(input, stack):
    # Note that input is a DataFrame
    dims = input.shape[1]
    output = []
    
    if dims == 1:
        # only the measure dim
        input_sum = sum( project_data(input, 0) )
        output = stack + [input_sum]
        return output
    else:
        # the general case
        dim0_vals = set(project_data(input, 0).values)
        dict1 = {}
        for dim0_v in dim0_vals:
            temp = stack + [dim0_v]
            sub_data = slice_data_dim0(input, dim0_v)
            temp = buc_rec(sub_data)
            output += [temp]
        ## for R_{ALL}
        temp = stack + ['ALL']
        sub_data = remove_first_dim(input)
        temp = buc_rec(sub_data)
        output += [temp]
    
    
    
    return output